# PIC-SURE API use-case: quick analysis on Hematopoietic Cell Transplant for Sickle Cell Disease (HCT for SCD) data

This is a tutorial notebook aimed to get the user quickly up and running with the R PIC-SURE API. It covers the main functionalities of the API.

 -------   

# Getting your own user-specific security token

**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

# Environment set-up

### Pre-requisites
- R 3.4 or later

### Install Packages

**Note that if you are using the dedicated PIC-SURE environment within the BioData Catalyst Seven Bridges platform, the necessary packages have already been installed.**

*Troubleshooting: if you encounter an error while installing packages or connecting to a PIC-SURE resource, try restarting your kernel and trying again.*

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")

In [ ]:
### Uncomment this code if you are not using the PIC-SURE environment in Seven Bridges, or if you do not have all the necessary dependencies installed.
#install.packages(c("devtools"))

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", ref="ALS-3799", force=T, quiet=FALSE)
library(dplyr)

## Connecting to a PIC-SURE resource

The following is required to get access to data through the PIC-SURE API: 
- Network URL
- Resource id
- User-specific security token

If you have not already retrieved your user-specific token, please refer to the "Get your security token" section of the [README.md](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token) file.

In [ ]:
# integration url
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure" 
#PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
token_file <- "token.txt"
token <- scan(token_file, what = "character")
session <- picsure::bdc.initializeSession(PICSURE_network_URL, token)
session <- picsure::bdc.setResource(session = session,  resourceName = "AUTH")

# Sickle Cell Example

Let's search the data dictionary in PIC-SURE to find all the variables in the `HCT for SCD` study, which is associated to PHS study number `phs002385`. 

In [ ]:
df <- picsure::bdc.searchPicsure(session, 'phs002385')

In [ ]:
head(df)

### Building a query: investigating male patients with avascular necrosis who received their transplant after 1999

Let's say we are interested in the age at which patients from the following cohort received their transplant:
* males
* patients with avascular necrosis
* patients who received their transplant after 1999

We will use regular expressions to search the variable descriptions within the HCT for SCD study to find these variables.

In [ ]:
variables_of_interest <- df[grepl('Sex|Avascular necrosis|Year of transplant$', df$var_description),]

sex_var <- df[grepl('Sex', df$var_description),'name']
avascular_necrosis_var <- df[grepl('Avascular necrosis', df$var_description),'name']
transplant_yr_var <- df[grepl('Year of transplant$', df$var_description),'name']

variables_of_interest

Now we can create a new query using the `name` associated with our variables of interest and apply our filters to retrieve the cohort of interest.

In [ ]:
myquery <- picsure::bdc.newQuery(session) # Initialize a new query

# Filter to Males
myquery <- picsure::bdc.addClause(query = myquery, 
                                  keys = sex_var,
                                  type = 'FILTER',
                                  categories = list('Male'))

# Filter to patients with Avascular Necrosis
myquery <- picsure::bdc.addClause(query = myquery, 
                                  keys = avascular_necrosis_var,
                                  type = 'FILTER',
                                  categories = list('Yes'))
                                  

# Filter to patients with year of transplant after 1999
myquery <- picsure::bdc.addClause(query = myquery, 
                                  keys = transplant_yr_var,
                                  type = 'FILTER',
                                  min = 1999, max = 2019)


We are also interested in the patients' age at which they received their transplant. However, we do not want to filter our cohort based on these values. We can use the query.select method to add this variable to the query without filtering.

Using this cohort, we can add the variable of interest: "Patient age at transplant, years"

In [ ]:
age_transplant_var <- df[grepl('age at transplant, years$', df$var_description),'name']
myquery <- picsure::bdc.addClause(query = myquery, 
                                  keys = age_transplant_var,
                                  type = 'SELECT')

## Retrieving the data

Once our query object is finally built, we use the `query.run` function to retrieve the data corresponding to our query

In [ ]:
results <-  picsure::runQuery(myquery)
head(results)

Once the data has been retrieved as a dataframe, you can use R functions to conduct analyses and create visualizations, such as this:

In [ ]:
hist(results[,5], breaks = 10, 
     xlab = 'Age received transplant, yrs old', 
     main = "Histogram of the ages at which males with Avascular Necrosis \nreceived their transplant in the HCT for SCD study")